In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/OffensEval_2019.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,text,class
0,0,user ask native americans take is,1
1,1,user user home you re drunk ! ! ! user maga t...,1
2,2,amazon investigating chinese employees selling...,0
3,3,user someone should have taken piece shit vol...,1
4,4,user user obama wanted liberals amp; illegals...,0


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

18944


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001, 0.001, 0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
9532/9532 [==============================] - 51s 5ms/step - loss: 0.5634 - acc: 0.7167
Epoch 2/3
9532/9532 [==============================] - 47s 5ms/step - loss: 0.4998 - acc: 0.7622
Epoch 3/3
9532/9532 [==============================] - 47s 5ms/step - loss: 0.4749 - acc: 0.7743
Epoch 1/3
9532/9532 [==============================] - 47s 5ms/step - loss: 0.5568 - acc: 0.7232
Epoch 2/3
9532/9532 [==============================] - 47s 5ms/step - loss: 0.4961 - acc: 0.7607
Epoch 3/3
9532/9532 [==============================] - 47s 5ms/step - loss: 0.4758 - acc: 0.7776
Epoch 1/3
9532/9532 [==============================] - 49s 5ms/step - loss: 0.5657 - acc: 0.7144
Epoch 2/3
9532/9532 [==============================] - 45s 5ms/step - loss: 0.4968 - acc: 0.7628
Epoch 3/3
9532/9532 [==============================] - 45s 5ms/step - loss: 0.4734 - acc: 0.7785
Epoch 1/3
9532/9532 [==============================] - 46s 5ms/step - loss: 0.5200 - acc: 0.7474
Epoch 2/3
9532/9532 [=========

In [8]:
print(score)

[[0.6652099742983792, 0.683598762382309, 0.6887618052596126], [0.7039580705047336, 0.6932979242676198, 0.6984445682529338], [0.6813249971554689, 0.7040112416964741, 0.7024015448694267]]


In [9]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_OffensEval_2019.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_OffensEval_2019.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_OffensEval_2019.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_OffensEval_2019.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
9532/9532 [==============================] - 50s 5ms/step - loss: 0.5275 - acc: 0.7448
Epoch 2/5
9532/9532 [==============================] - 50s 5ms/step - loss: 0.4596 - acc: 0.7937
Epoch 3/5
9532/9532 [==============================] - 50s 5ms/step - loss: 0.4191 - acc: 0.8104
Epoch 4/5
9532/9532 [==============================] - 50s 5ms/step - loss: 0.4153 - acc: 0.8156
Epoch 5/5
9532/9532 [==============================] - 54s 6ms/step - loss: 0.4007 - acc: 0.8234
Saved model to disk
Fold Number: 1
Epoch 1/5
9533/9533 [==============================] - 50s 5ms/step - loss: 0.5262 - acc: 0.7532
Epoch 2/5
9533/9533 [==============================] - 51s 5ms/step - loss: 0.4611 - acc: 0.7918
Epoch 3/5
9533/9533 [==============================] - 50s 5ms/step - loss: 0.4177 - acc: 0.8150
Epoch 4/5
9533/9533 [==============================] - 55s 6ms/step - loss: 0.3706 - acc: 0.8347
Epoch 5/5
9533/9533 [==============================] - 47s 5ms/step - loss: 0

In [10]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.7487416107382551, 0.7582878724297104, 0.7448594208980277, 0.7293327738145196, 0.7624842635333613]
Mean: 0.7487411882827747
Std Dev: 0.011591082485462428


In [11]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.7147463498229272, 0.717128485485969, 0.7238702201622248, 0.6894485422698997, 0.7297822195082468]
Mean: 0.7149951634498535
Std Dev: 0.01381786150867534


In [5]:
# load json and create model
json_file = open('Models/model_OffensEval_2019.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_OffensEval_2019.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f7411c7c4e0> False
<keras.layers.recurrent.LSTM object at 0x7f7411c7c5f8> True
<keras.layers.core.Dense object at 0x7f7411c7c5c0> True
<keras.layers.core.Dense object at 0x7f7411b0bef0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f7411c7c4e0> False
<keras.layers.recurrent.LSTM object at 0x7f7411c7c5f8> False
<keras.layers.core.Dense object at 0x7f7411c7c5c0> True
<keras.layers.core.Dense object at 0x7f7411b0bef0> True
Epoch 1/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.4314 - acc: 0.8036
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.3242 - acc: 0.8330
Epoch 3/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2908 - acc: 0.8469
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2727 - acc: 0.8587
Epoch 5/8
4460/4460 [==============================] - 7s 2ms/step - loss: 

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8702375616315553, 0.868668758404303, 0.8730941704035874, 0.8809417040358745, 0.8804932735426009]
Mean: 0.8746870936035842
Std Dev: 0.00512616766807439
Macro-F1-Scores: 
[0.7630566785188861, 0.7375222451365868, 0.7596649651392525, 0.7675527255327763, 0.76199618121402]
Mean: 0.7579585591083043
Std Dev: 0.010534666520271569


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.7727476467951592, 0.7642312864186463, 0.7672645739910314, 0.7677130044843049, 0.7688340807174888]
Mean: 0.7681581184813261
Std Dev: 0.00275453275032857
Macro-F1-Scores: 
[0.6885486131934342, 0.679924044059246, 0.6835444595466973, 0.6827570372713931, 0.6859738143740328]
Mean: 0.6841495936889607
Std Dev: 0.002927108610489425


In [10]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1


In [11]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f7418d3f080> False
<keras.layers.recurrent.LSTM object at 0x7f7418d3f198> True
<keras.layers.core.Dense object at 0x7f7418d3f160> True
<keras.layers.core.Dense object at 0x7f741848ddd8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f7418d3f080> False
<keras.layers.recurrent.LSTM object at 0x7f7418d3f198> False
<keras.layers.core.Dense object at 0x7f7418d3f160> True
<keras.layers.core.Dense object at 0x7f741848ddd8> True
Epoch 1/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.7004 - acc: 0.6906
Epoch 2/8
2841/2841 [==============================] - 4s 2ms/step - loss: 0.5996 - acc: 0.7001
Epoch 3/8
2841/2841 [==============================] - 4s 2ms/step - loss: 0.5643 - acc: 0.7181
Epoch 4/8
2841/2841 [==============================] - 4s 2ms/step - loss: 0.5459 - acc: 0.7314
Epoch 5/8
2841/2841 [==============================] - 4s 2ms/step - loss: 

In [12]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7622230038691523, 0.7610837438423645, 0.7568613652357494, 0.7596762843068262, 0.7539598732840549]
Mean: 0.7587608541076295
Std Dev: 0.0029959233963263686
Macro-F1-Scores: 
[0.6949621168353114, 0.6923926185756653, 0.6924012622715845, 0.6758881665016123, 0.6774016480325544]
Mean: 0.6866091624433455
Std Dev: 0.008203495693137164


In [13]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6841364755539923, 0.6885995777621393, 0.687192118226601, 0.6924700914848698, 0.7043294614572334]
Mean: 0.6913455448969672
Std Dev: 0.007022943309977718
Macro-F1-Scores: 
[0.6094875795398923, 0.6208989197932101, 0.6158139049705315, 0.6213058419243986, 0.6389433551198257]
Mean: 0.6212899202695716
Std Dev: 0.009808964136324913


In [14]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0


In [15]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f7395c84240> False
<keras.layers.recurrent.LSTM object at 0x7f7392832f28> True
<keras.layers.core.Dense object at 0x7f7395c84080> True
<keras.layers.core.Dense object at 0x7f7395c8f668> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f7395c84240> False
<keras.layers.recurrent.LSTM object at 0x7f7392832f28> False
<keras.layers.core.Dense object at 0x7f7395c84080> True
<keras.layers.core.Dense object at 0x7f7395c8f668> True
Epoch 1/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.9787 - acc: 0.6545
Epoch 2/8
1618/1618 [==============================] - 2s 2ms/step - loss: 0.5417 - acc: 0.7503
Epoch 3/8
1618/1618 [==============================] - 2s 2ms/step - loss: 0.4443 - acc: 0.8201
Epoch 4/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4238 - acc: 0.8276
Epoch 5/8
1618/1618 [==============================] - 2s 2ms/step - loss: 

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8326127239036443, 0.8288009888751545, 0.8288009888751545, 0.826946847960445, 0.8312731767614339]
Mean: 0.8296869452751665
Std Dev: 0.002007699264853745
Macro-F1-Scores: 
[0.5288151028662068, 0.5617855693690084, 0.5262680020082975, 0.5005402068220405, 0.5087716566781917]
Mean: 0.5252361075487489
Std Dev: 0.021122039807656205


In [17]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6213712168004941, 0.5976514215080346, 0.619283065512979, 0.6081582200247219, 0.6032138442521632]
Mean: 0.6099355536196785
Std Dev: 0.00913663575114911
Macro-F1-Scores: 
[0.5436893114605158, 0.5164617337394501, 0.5358473414401526, 0.5316371778388171, 0.5279686341229333]
Mean: 0.5311208397203737
Std Dev: 0.009004428840641303


In [18]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link


In [19]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f738efcf630> False
<keras.layers.recurrent.LSTM object at 0x7f738efcf3c8> True
<keras.layers.core.Dense object at 0x7f738efdc5c0> True
<keras.layers.core.Dense object at 0x7f7388107668> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f738efcf630> False
<keras.layers.recurrent.LSTM object at 0x7f738efcf3c8> False
<keras.layers.core.Dense object at 0x7f738efdc5c0> True
<keras.layers.core.Dense object at 0x7f7388107668> True
Epoch 1/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3864 - acc: 0.8505
Epoch 2/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3481 - acc: 0.8634
Epoch 3/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3287 - acc: 0.8759
Epoch 4/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3160 - acc: 0.8832
Epoch 5/8
1926/1926 [==============================] - 3s 2ms/step - loss: 

In [20]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8843360995850622, 0.887331256490135, 0.8811007268951194, 0.8909657320872274, 0.8909657320872274]
Mean: 0.8869399094289543
Std Dev: 0.003832584690274088
Macro-F1-Scores: 
[0.4947393161639843, 0.5197454527486898, 0.5575598349629389, 0.514488752310758, 0.5405701355821309]
Mean: 0.5254206983537004
Std Dev: 0.0217048041703208


In [21]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8319502074688797, 0.8161993769470405, 0.818276220145379, 0.828141225337487, 0.8265835929387332]
Mean: 0.8242301245675039
Std Dev: 0.006006281508934312
Macro-F1-Scores: 
[0.6185954500494559, 0.6077663870767319, 0.6071064711738197, 0.6396290164028022, 0.6225789418101173]
Mean: 0.6191352533025853
Std Dev: 0.01188349631093031


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0


In [23]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f737ec4be48> False
<keras.layers.recurrent.LSTM object at 0x7f737b7fc390> True
<keras.layers.core.Dense object at 0x7f737ec4b320> True
<keras.layers.core.Dense object at 0x7f73991de358> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f737ec4be48> False
<keras.layers.recurrent.LSTM object at 0x7f737b7fc390> False
<keras.layers.core.Dense object at 0x7f737ec4b320> True
<keras.layers.core.Dense object at 0x7f73991de358> True
Epoch 1/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.7857 - acc: 0.7061
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2304 - acc: 0.9419
Epoch 3/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2119 - acc: 0.9424
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2048 - acc: 0.9426
Epoch 5/8
4460/4460 [==============================] - 7s 2ms/step - loss: 

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.9421783953384133, 0.9428507395786643, 0.9405829596412556, 0.9412556053811659, 0.9412556053811659]
Mean: 0.941624661064133
Std Dev: 0.0007959191188799695
Macro-F1-Scores: 
[0.4889569571702535, 0.5182288524098719, 0.4921187958157705, 0.5032343418403439, 0.5235633346163129]
Mean: 0.5052204563705105
Std Dev: 0.013753393154509424


In [25]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.32765575974899147, 0.3565665620797849, 0.34080717488789236, 0.33923766816143497, 0.35672645739910314]
Mean: 0.3441987244554413
Std Dev: 0.011132761288288944
Macro-F1-Scores: 
[0.2858150469048678, 0.3043010603780932, 0.29333260901057506, 0.2933093483133429, 0.3067942532097579]
Mean: 0.29671046356332736
Std Dev: 0.0077587056152744575


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f628653b2b0> False
<keras.layers.recurrent.LSTM object at 0x7f628653b3c8> True
<keras.layers.core.Dense object at 0x7f628653b390> True
<keras.layers.core.Dense object at 0x7f6283773d68> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f628653b2b0> False
<keras.layers.recurrent.LSTM object at 0x7f628653b3c8> False
<keras.layers.core.Dense object at 0x7f628653b390> True
<keras.layers.core.Dense object at 0x7f6283773d68> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8194375925012334, 0.819078947368421, 0.8141141635137358, 0.8027636124362559, 0.8073696331633492]
Mean: 0.8125527897965992
Std Dev: 0.006559302324113608
Macro-F1-Scores: 
[0.8148323519754358, 0.8143136418059064, 0.8092267524620973, 0.7980989718553488, 0.8032273711002698]
Mean: 0.8079398178398115
Std Dev: 0.006462789853406684
Without Fine Tuning: 
Accuracies: 
[0.7556322973195199, 0.7610197368421052, 0.7611449251521631, 0.74814936667215, 0.7511103799967099]
Mean: 0.7554113411965296
Std Dev: 0.005207906480285314
Macro-F1-Scores: 
[0.7553542649478555, 0.7608077557697469, 0.7609557752200407, 0.7480916695837795, 0.7508919720444059]
Mean: 0.7552202875131657
Std Dev: 0.005170784835734205


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f620881ff60> False
<keras.layers.recurrent.LSTM object at 0x7f620881fd68> True
<keras.layers.core.Dense object at 0x7f620881fa90> True
<keras.layers.core.Dense object at 0x7f6277a28d30> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f620881ff60> False
<keras.layers.recurrent.LSTM object at 0x7f620881fd68> False
<keras.layers.core.Dense object at 0x7f620881fa90> True
<keras.layers.core.Dense object at 0x7f6277a28d30> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8225926847474496, 0.8280238924838228, 0.8315082130413141, 0.818815331010453, 0.8282727725236436]
Mean: 0.8258425787613367
Std Dev: 0.004532271067059321
Macro-F1-Scores: 
[0.7123585041361777, 0.7217728968783055, 0.7316556513253578, 0.704663823462893, 0.7200797227876321]
Mean: 0.7181061197180731
Std Dev: 0.00910670724340095
Without Fine Tuning: 
Accuracies: 
[0.6698183627768102, 0.660776505724241, 0.6752115480338476, 0.6575410652065704, 0.6704828272772524]
Mean: 0.6667660618037443
Std Dev: 0.006564030297393381
Macro-F1-Scores: 
[0.6407068354259826, 0.6337756953086199, 0.6461768708749938, 0.6286906501336071, 0.639610691381302]
Mean: 0.637792148624901
Std Dev: 0.006017893919725546


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_OffensEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    OffensEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    OffensEval_2019_model.load_weights("Models/model_OffensEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in OffensEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in OffensEval_2019_model.layers:
        print(layer, layer.trainable)
    OffensEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    OffensEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=OffensEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fee94b221d0> False
<keras.layers.recurrent.LSTM object at 0x7fee94b222e8> True
<keras.layers.core.Dense object at 0x7fee94b222b0> True
<keras.layers.core.Dense object at 0x7fee94b7bf60> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fee94b221d0> False
<keras.layers.recurrent.LSTM object at 0x7fee94b222e8> False
<keras.layers.core.Dense object at 0x7fee94b222b0> True
<keras.layers.core.Dense object at 0x7fee94b7bf60> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8382936507936508, 0.8391261171797418, 0.8449304174950298, 0.8469184890656064, 0.8399602385685885]
Mean: 0.8418457826205235
Std Dev: 0.0034297760855209755
Macro-F1-Scores: 
[0.7768284881075436, 0.7784468709256844, 0.769299973539529, 0.7849579976569744, 0.7794301293600633]
Mean: 0.777792691917959
Std Dev: 0.005049885391330454
Without Fine Tuning: 
Accuracies: 
[0.7490079365079365, 0.7428003972194638, 0.7554671968190855, 0.7415506958250497, 0.7395626242544732]
Mean: 0.7456777701252018
Std Dev: 0.005824601433052034
Macro-F1-Scores: 
[0.7091871119296185, 0.7036529094084375, 0.7095474839091305, 0.7035932350901368, 0.6972733328739117]
Mean: 0.7046508146422469
Std Dev: 0.0044965962592939295
